# Import and Init global parameters

In [1]:
import json
from os.path import join
from datetime import datetime, date
import sys
import git

dev_path = join('c:\\', 'dev', 'dyn_dev')  # path to development folder with project code (where git configured)
distr_path = join('c:\\', 'dev', 'distr')  # path to folder where distr will be built
load_file_path = join('c:\\', 'dev', 'test_data')  # path to folder with files to load into app

use_local_pip = False  # is downlod pip packages into each distr and use it in deploy
is_verbose = True  # print a lot of debug information

# fact dates for each SB on which MACRO will be loaded
sb_fact_macro_load_date = {
    'sw': [date(2020, 1, 31), date(2020, 2, 29), date(2020, 3, 31), date(2020, 4, 30)],
    'kz': [date(2019, 12, 31), date(2020, 1, 31), date(2020, 2, 29)],
    'by': [date(2019, 12, 31), date(2020, 1, 31)],
}

# fact dates for each SB on which DEAL will be loaded
sb_fact_deal_load_date = {
    'sw': [date(2020, 1, 31), date(2020, 2, 29), date(2020, 3, 31), date(2020, 4, 30)],
    'kz': [date(2020, 2, 28)],
    'by': [date(2020, 1, 31)],
}

# end of forecast periods for creating all solves (including budget solve)
sb_forecast_till = {
    'sw': date(2020, 12, 31),
    'kz': date(2020, 12, 31),
    'by': date(2020, 12, 31),
}

# releases that are already deployed (current) in each SB and releases that are going to be deployed (new)
# allowed use one of: git tag (e.d. 8.0), dev, master
sb_release = {
    'sw': {'current': '11.0', 'new': 'dev'},
    'kz': {'current': '13.1', 'new': 'dev'},
    'by': {'current': '13.0', 'new': 'dev'},
    'cet': {'current': 'dev', 'new': 'dev'},
}

# Build releases for all used tags

In [2]:
# get all used tags from sb_release
release_set = set()
for release_dict in sb_release.values():
    release_set.update({release_dict['current'], release_dict['new']})
    
# preserve release names for every release
release_dict = dict()

# preserve active branch
repo = git.Repo(dev_path)
current_branch = repo.active_branch
print(f'Preserve current branch "{current_branch}"\n')
current_sys_path = sys.path

try:
    autobuild_lib_path = join(dev_path, 'scripts')
    sys.path.append(autobuild_lib_path)

    for git_tag in release_set:
        from automate_build import AutomateBuild
        AutomateBuild.git_checkout(repo_path=dev_path, git_tag=git_tag)

        # reload lib from checkouted code
        del AutomateBuild
        from automate_build import AutomateBuild

        # build release
        AutomateBuild.build_distr(dev_path=dev_path, distr_path=distr_path, git_tag=git_tag, use_local_pip=use_local_pip, is_verbose=is_verbose)
        release_dict[git_tag] = AutomateBuild._get_release_name(git_tag)
        print()
        
finally:    
    sys.path = current_sys_path
    del AutomateBuild
    # revert to active branch
    checkout_result = repo.git.checkout(current_branch)
    print(f'Reverted current branch to "{current_branch}". Git info "{checkout_result}"')
    print(f'Succesfully built distrs: "{release_dict}"')

Preserve current branch "dev"

Git checkouted to "dev"
Starting "['scripts\\build_distr.bat', 'c:\\dev\\distr', 'dynbal_sb_dev']"...
Deleted existing distr files
Copied distr files
Скопировано файлов: 36.
Скопировано файлов: 883.
Скопировано файлов: 4.
Скопировано файлов: 17.
Скопировано файлов: 86.
Скопировано файлов: 9.
Copied distr files
Deleted test data
Distributive succesfully built!

Finished "['scripts\\build_distr.bat', 'c:\\dev\\distr', 'dynbal_sb_dev']"!

Git checkouted to "13.0"
Starting "['scripts\\build_distr.bat', 'c:\\dev\\distr', 'dynbal_sb_13_0']"...
Deleted existing distr files
Copied distr files
Скопировано файлов: 33.
Скопировано файлов: 859.
Скопировано файлов: 4.
Скопировано файлов: 17.
Скопировано файлов: 86.
Скопировано файлов: 9.
Copied distr files
Deleted test data
Distributive succesfully built!

Finished "['scripts\\build_distr.bat', 'c:\\dev\\distr', 'dynbal_sb_13_0']"!

Git checkouted to "11.0"
Starting "['scripts\\build_distr.bat', 'c:\\dev\\distr', 'dyn

# Deploy and load data into app

In [8]:
for sb_code in ('kz', ): # ('sw', 'by', 'kz', ):
    # clear db and deploy current release on empty db + initially fill app
    current_sys_path = sys.path
    current_release_name = release_dict[sb_release[sb_code]['current']]
    autobuild_lib_path = join(distr_path, current_release_name, 'scripts')
    sys.path.append(autobuild_lib_path)        
    try:
        from automate_build import AutomateBuild as CurrentAutomateBuild
        current_builder = CurrentAutomateBuild(sb_code=sb_code, distr_path=distr_path, git_tag=sb_release[sb_code]['current'])
        
        current_builder.clear_db()
        
        current_builder.deploy_distr()
        
        current_load_file_path = join(load_file_path, current_release_name)
        current_builder.fill_app(
            load_file_path=current_load_file_path, 
            fact_macro_load_date=sb_fact_macro_load_date[sb_code], 
            fact_deal_load_date=sb_fact_deal_load_date[sb_code], 
            forecast_till=sb_forecast_till[sb_code], 
            fill_type='init',
        )
    finally:
        sys.path = current_sys_path
        del CurrentAutomateBuild
    
    # deploy new release upon existing app with previous release
    # restart existing solve, reload fact and run new solves
    current_sys_path = sys.path
    new_release_name = release_dict[sb_release[sb_code]['new']]
    autobuild_lib_path = join(distr_path, new_release_name, 'scripts')
    sys.path.append(autobuild_lib_path)        
    try:
        from automate_build import AutomateBuild as NewAutomateBuild
        new_builder = NewAutomateBuild(sb_code=sb_code, distr_path=distr_path, git_tag=sb_release[sb_code]['new'])
        
        new_builder.deploy_distr()
        
        new_load_file_path = join(load_file_path, new_release_name)
        new_builder.fill_app(
            load_file_path=new_load_file_path, 
            fact_macro_load_date=sb_fact_macro_load_date[sb_code], 
            fact_deal_load_date=sb_fact_deal_load_date[sb_code], 
            forecast_till=sb_forecast_till[sb_code], 
            fill_type='reload',
        )
    finally:
        sys.path = current_sys_path
        del NewAutomateBuild

Set sb_config="preprod_kz"
Set release_name="dynbal_sb_13_1"
Set release_path="c:\dev\distr\dynbal_sb_13_1"
Starting "['python', 'manage.py', 'clear_db_to_zero', '--settings=config.settings.preprod_kz', '--no-confirm']"...
Migrations have been deleted

Finished "['python', 'manage.py', 'clear_db_to_zero', '--settings=config.settings.preprod_kz', '--no-confirm']"!
Starting "['scripts\\deploy.bat', 'preprod_kz', 'y']"...

c:\dev\distr\dynbal_sb_13_1>ECHO Deploying Dynamic Balance ...  1>starts_preprod_kz_2021-03-03_18-45-30.log 

c:\dev\distr\dynbal_sb_13_1>IF [n] == [y] pip.exe install -r requirements\prod.txt --no-index --find-links=.\_pypi_distr  1>>starts_preprod_kz_2021-03-03_18-45-30.log 

c:\dev\distr\dynbal_sb_13_1>IF [n] == [n] pip.exe install -r requirements\prod.txt  1>>starts_preprod_kz_2021-03-03_18-45-30.log 

c:\dev\distr\dynbal_sb_13_1>python manage.py collectstatic --noinput  1>>starts_preprod_kz_2021-03-03_18-45-30.log 

c:\dev\distr\dynbal_sb_13_1>python manage.py migr

# Useful debug

In [44]:
# build using dev_path insteaf of distr path as usual (to simplify debug)
sb_code = 'sw'
builder = AutomateBuild(sb_code=sb_code, dev_path=dev_path, distr_path=distr_path, git_tag=sb_release[sb_code]['new'])
builder.release_name = 'dyn_dev'
builder.release_path = dev_path

builder.clear_db()
builder.deploy_distr(use_local_pip=False)
builder.fill_app(
    load_file_path=load_file_path, 
    fact_macro_load_date=sb_fact_macro_load_date[sb_code], 
    fact_deal_load_date=sb_fact_deal_load_date[sb_code], 
    forecast_till=sb_forecast_till[sb_code], 
    fill_type='init',
)

Set sb_config="preprod_sw"
Set release_name="dynbal_sb_dev"
Set release_path="c:\dev\distr\dynbal_sb_dev"
Starting "['python', 'manage.py', 'clear_db_to_zero', '--settings=config.settings.preprod_sw', '--no-confirm']"...
Migrations have been deleted

Finished "['python', 'manage.py', 'clear_db_to_zero', '--settings=config.settings.preprod_sw', '--no-confirm']"!
Starting "['scripts\\deploy.bat', 'preprod_sw']"...

c:\dev\dyn_dev>ECHO Deploying Dynamic Balance ...  1>starts_preprod_sw_2020-06-03_14-05-23.log 

c:\dev\dyn_dev>IF [n] == [y] pip.exe install -r requirements\prod.txt --no-index --find-links=.\_pypi_distr  1>>starts_preprod_sw_2020-06-03_14-05-23.log 

c:\dev\dyn_dev>IF [n] == [n] pip.exe install -r requirements\prod.txt  1>>starts_preprod_sw_2020-06-03_14-05-23.log 

c:\dev\dyn_dev>python manage.py collectstatic --noinput  1>>starts_preprod_sw_2020-06-03_14-05-23.log 

c:\dev\dyn_dev>python manage.py migrate  1>>starts_preprod_sw_2020-06-03_14-05-23.log 

c:\dev\dyn_dev>ECHO 

In [10]:
sb_code = 'sw'
autobuild_lib_path = join(dev_path, 'scripts')
sys.path.append(autobuild_lib_path)
from automate_build import AutomateBuild
builder = AutomateBuild(sb_code=sb_code, distr_path=distr_path, git_tag=sb_release[sb_code]['new'])
builder.release_name = 'dyn_dev'
builder.release_path = dev_path

cmd = ['workon', 'dynbal_sb', ';', 'python', '--version']
builder._run_cmd(cmd, cwd=dev_path)

Set sb_config="preprod_sw"
Set release_name="dynbal_sb_dev"
Set release_path="c:\dev\distr\dynbal_sb_dev"
Starting "['workon', 'dynbal_sb', ';', 'python', '--version']"...

Finished "['workon', 'dynbal_sb', ';', 'python', '--version']"!


In [3]:
import subprocess
python_path = 'c:\\dev\\venvs\\dynbal_sb\\Scripts\\python'
cmd = [python_path, 'manage.py', 'showmigrations', '--settings=config.settings.preprod_sw']
result = subprocess.run(cmd, shell=True, cwd=dev_path, encoding='cp866',  stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(result.stderr)
print(result.stdout)


admin
 [X] 0001_initial
 [X] 0002_logentry_remove_auto_add
 [X] 0003_logentry_add_action_flag_choices
auth
 [X] 0001_initial
 [X] 0002_alter_permission_name_max_length
 [X] 0003_alter_user_email_max_length
 [X] 0004_alter_user_username_opts
 [X] 0005_alter_user_last_login_null
 [X] 0006_require_contenttypes_0002
 [X] 0007_alter_validators_add_error_messages
 [X] 0008_alter_user_username_max_length
 [X] 0009_alter_user_last_name_max_length
base
 [X] 0001_initial
 [X] 0002_update_ref
 [X] 0003_auto_20190827_1410
 [X] 0004_auto_20190930_1708
 [X] 0005_auto_20191014_1710
 [X] 0006_auto_20200312_1655
contenttypes
 [X] 0001_initial
 [X] 0002_remove_content_type_name
django_cron
 [X] 0001_initial
 [X] 0002_remove_max_length_from_CronJobLog_message
input
 [X] 0001_initial
 [X] 0002_main_mis_product_code
 [X] 0003_auto_20190814_1735
 [X] 0004_auto_20190819_1305
 [X] 0005_auto_20190819_1305
 [X] 0006_auto_20190819_1330
 [X] 0007_remove_budget_solve
 [X] 0008_auto_20190819_1841
 [X] 0009_auto_20